In [1]:
import os, sys
from shutil import copyfile
from datetime import datetime
import importlib
import skimage

In [2]:
import fastai
from fastai.vision.all import *
from fastai.basics import *
from torch.utils.tensorboard import SummaryWriter
from fastai.callback.fp16 import ModelToHalf
from fastai.callback.hook import hook_output
import tensorboard

In [3]:
from tensorboard_helpers import TensorBoardBaseCallback, TensorBoardCallback

In [9]:
size = 64 # Progressive resizing could happen here
augs = aug_transforms(flip_vert=True, max_warp=0.1, size=size)

## CV2 Parser

In [62]:
from ceruleanml import data
import icevision

In [11]:
class_map = {v: k for k, v in data.class_mapping_coco_inv.items()}

In [12]:
class_map

{'Infrastructure': 1,
 'Natural Seep': 2,
 'Coincident Vessel': 3,
 'Recent Vessel': 4,
 'Old Vessel': 5,
 'Ambiguous': 6,
 'Hard Negatives': 0}

#### img tile fnames

In [65]:
parser = icevision.parsers.COCOMaskParser(annotations_filepath="../../data/cv2_transfer/instances_slicks_test_v2.json", img_dir="../../data/tiled_image_slicks_test_v2")
train_records = parser.parse(data_splitter=icevision.data.SingleSplitSplitter())

  0%|          | 0/487 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/487 [00:00<?, ?it/s]

In [91]:
tr.as_dict().keys()

dict_keys(['common', 'detection'])

In [92]:
sparse_masks = {}
for tr in train_records[0]:
    sparse_masks.update({tr.as_dict()['common']['filepath']:tr.as_dict()['detection']['masks']})

need to build a fastai data loader that doesn't take img paths and instead tkaes in mem sparse arrays

In [96]:
def get_chps(path): return list(sparse_masks.keys())
def get_lbls(fn): return sparse_masks[fn] # fn is the imge path 


#def splitter_func(fn): return fn.name.rsplit('_',1)[0] in valid_names # XXX Check to make sure this should be returning True for Valid


In [ ]:
coco_seg_dblock = DataBlock(blocks=(ImageBlock, MaskBlock(codes=codes)),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=[lambda o: train_records[o.name][0], lambda o: img2bbox[o.name][1]], 
                 item_tfms=Resize(128),
                 batch_tfms=aug_transforms(),
                 n_inp=1)

## CV1 Parser

In [18]:
mount_path = "/root/data/cv1_transfer/"
ml_data_path = os.path.join(mount_path, "labeled_data")
path = Path(ml_data_path)
oil_chps = np.loadtxt(path/'oil_chps.txt', dtype=str)
codes = np.loadtxt(path/'codes.txt', dtype=str)
valid_names = np.loadtxt(path/'valid_names.txt', dtype=str)

In [19]:
import skimage.io as skio
import numpy as np
import dask

labels = path/"lbl"
labels_no_3 = path/"lbl_no3"

os.makedirs(labels_no_3, exist_ok=True)

vals = set()
lazy_results = []
for i in list(labels.glob("*.png"))[0:100]:
    arr = skio.imread(i)
    values = np.unique(arr)
    # I think this was setting a class we didn't need to 0 
    # for the purposes of recreating Jona's binary classifier
    arr[arr==3] = 0
    try:
        lazy_result = dask.delayed(skio.imsave)(labels_no_3 / i.name, arr, check_contrast=False)
        lazy_results.append(lazy_result)
    except:
        continue
computed_results = dask.compute(*lazy_results)

In [20]:
random.seed(42)
oilless_chps = [chp for chp in get_image_files(path/'chp') if chp.stem not in oil_chps]
density = 0
num_oilless = density * len(oil_chps)
mixed_chps = random.sample(oilless_chps,num_oilless) + [path/'chp'/(chp+'.png') for chp in oil_chps]
len(mixed_chps)

473

In [ ]:
def get_chps(path): return mixed_chps
def get_lbls(fn): return fn.parent.parent/"lbl_no3"/f"{fn.name}"
def splitter_func(fn): return fn.name.rsplit('_',1)[0] in valid_names # XXX Check to make sure this should be returning True for Valid

drive_files = get_chps(path)

In [ ]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y_%H_%M_%S")
lcl =  Path('/root/data/experiments/cv1/'+timestampStr+'/')
#lcl = Path("../out_data")
lcl.mkdir(exist_ok=True)
modelpath = lcl

use_lcl = False
if use_lcl:
  path = lcl
  lcl_chp = path/"chp"
  lcl_lbl = path/"lbl_no3"
  lcl_chp.mkdir(exist_ok=True)
  lcl_lbl.mkdir(exist_ok=True)
  for i, f in enumerate(drive_files): # 10 minutes?!
    if not i%50: print(i, f)
    if not (lcl_chp/f.name).exists():
      copyfile(f, lcl_chp/f.name)
    if not (lcl_lbl/f.name).exists():
      copyfile(get_lbls(f), lcl_lbl/f.name)

In [ ]:
seg_db = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes=codes)),
    get_items = get_chps,
    splitter=FuncSplitter(splitter_func),
    batch_tfms=augs,
    get_y=get_lbls)

In [ ]:
dls = SegmentationDataLoaders.from_dblock(
    dblock = seg_db,
    source = path,
    path = path,
    bs = 4)

In [ ]:
seg_db.summary(path)

In [ ]:
dls.show_batch(vmin=0, vmax=3)

In [ ]:
# add best model callback saver
# write to drive not lcl

In [ ]:
arch = 18
archs = {18: resnet18, 34: resnet34, 50: resnet50}

In [ ]:
learn = unet_learner(dls, archs[arch], metrics=[Dice()], model_dir=modelpath, cbs=[MixedPrecision])

In [ ]:
lr = learn.lr_find()

In [ ]:
lr

In [ ]:
cbs = [TensorBoardCallback(projector=False, trace_model=False)]


In [ ]:
learn.fine_tune(5, 2e-4, cbs=cbs)#, cbs=SaveModelCallback(monitor='dice'))w

In [ ]:
# learn.load("model")

In [ ]:
learn

In [ ]:
savename = f'{density}_{arch}_{size}_{round(learn.validate()[1],3)}.pkl'
learn.export(f'{modelpath}/{savename}')

In [ ]:
ls {modelpath}

In [ ]:
learn.show_results(max_n=4, figsize=(20,20), vmin=0, vmax=3)

Default path for tensorboard logs is `./runs/`

In [ ]:
!ls './runs/'

Copy logs to appropriate exeriments folder in the mounted GCS volume.

In [ ]:
!cp -R './runs/' {modelpath}'/tensorboard/'

Now, run the following from anywhere with gcs authenticated:

In [ ]:
!tensorboard --logdir="/root/data/experiments/cv1/09_Mar_2022_18_32_17/tensorboard"
